In [1]:
from amm import amm
import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt
# plt.style.use('paper.mplstyle')

import seaborn as sns
sns.set_theme(style="ticks")

import statsmodels.api as sm
import matplotlib.ticker as mtick
import pickle

In [2]:
T = 120
Rx = 1000.
Ry = 1000.
batch_size = 100

kappa = np.array([0.2, 1., 5.])
sigma = np.array([0.2, 0.5, 1.])
p = np.array([0.2, 0.5, 0.8])
phi = np.array([0.01, 0.01, 0.01])

In [3]:
''' Initial reserves '''
Rx0   = np.repeat(Rx, len(kappa))
Ry0   = np.repeat(Ry, len(kappa))

pools = amm(Rx=Rx0, Ry=Ry0, phi=phi)

xs_0 = np.repeat(1., len(kappa))
l = pools.swap_and_mint(xs_0)
end_pools, Rx_t, Ry_t, v_t, event_type_t, event_direction_t = pools.simulate(kappa, p, sigma, T=T, batch_size=batch_size)

""" Burn and swap all coins into x """
x_T = np.zeros(batch_size)
for k in range(batch_size):
    x_T[k] = np.sum(end_pools[k].burn_and_swap(l))

x_0      = np.sum(xs_0)
log_ret  = np.log(x_T) - np.log(x_0) # performance / log return

100%|██████████| 100/100 [00:03<00:00, 30.31it/s]


In [4]:
log_ret = np.log(np.array([pool.Rx / pool.Ry for pool in end_pools]))

# Portfolio optimization